In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import spacy
import numpy as np
import scipy
from spacytextblob.spacytextblob import SpacyTextBlob
from operator import itemgetter
from itertools import groupby

In [2]:
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('spacytextblob')

In [3]:
f = open("beastquestnovel.txt", "r", encoding='utf8')

novel_book =''

lines = []
for line in f:
    stripped_line = line.rstrip() + ' '
    novel_book += stripped_line
    lines.append(line)
f.close()

doc = nlp(novel_book)

In [4]:
chunks = []
lines = []
for s in doc.sents:
    lines.append(s.text)
    if len(lines) > 10:
        chunks.append('\n'.join(lines))
        lines = []

In [5]:
n_features = 2000 # number of most common words
n_topics = 20 # number of topics
max_df = 0.95 # maximum document frequency
min_df = 0.001 # minimum document frequency

In [6]:
transformer = TfidfVectorizer(max_df=max_df, min_df=min_df, max_features=n_features, stop_words='english')
X = transformer.fit_transform(chunks)

In [7]:
nmf = NMF(n_components=n_topics, verbose=1, max_iter=100).fit(X)

c:\users\randika\appdata\local\programs\python\python39\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


violation: 1.0
violation: 0.285815358940067
violation: 0.19864611269739477
violation: 0.15025120180713852
violation: 0.12562472339763717
violation: 0.10730408303045412
violation: 0.09147518687125664
violation: 0.07681257088856061
violation: 0.0671778525780806
violation: 0.06116813553702858
violation: 0.05689299601196235
violation: 0.05378656013816708
violation: 0.049936679725670535
violation: 0.04667101415739945
violation: 0.04331023926564781
violation: 0.039902175501916305
violation: 0.036648476695129
violation: 0.03337160522082834
violation: 0.030755279478849207
violation: 0.02863139758658248
violation: 0.02642988210532604
violation: 0.024406265305979474
violation: 0.02315190112328007
violation: 0.0219243931283082
violation: 0.021124212156700628
violation: 0.020384433583209224
violation: 0.01953752333917073
violation: 0.018790985355546398
violation: 0.018336047263987455
violation: 0.018043371847589883
violation: 0.017643364548727565
violation: 0.017217479623129306
violation: 0.016729

c:\users\randika\appdata\local\programs\python\python39\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


In [8]:
nmf.verbose = 0

In [9]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [11]:
topic_sent_list = []

for s in doc.sents:
    topic_id = np.argmax(nmf.transform(transformer.transform([s.text]))[0]) 
    topic_sent_list.append([topic_id,s])
print(len(topic_sent_list))

1116


In [12]:
def get_sents_topic(id):
    temp = []
    for s in topic_sent_list:
        if s[0] == id:
            temp.append(s)
    return temp

temp = get_sents_topic(2)
print(len(temp))
for i in temp:
    doc = nlp(i[1].text)
    #print(i[1],doc._.polarity)
    if(doc._.polarity <0):
        print(i[1],doc._.polarity)

74
The knight’s bronze armor gleamed in the pale morning sunlight. -0.21
He stumbled to one side and a huge shudder threw him to the ground, knocking his head so badly that his teeth clattered. -0.14999999999999988
Was it about to come crashing down? -0.15555555555555559
Its eyes glinted with dark fire. -0.15
“Come down, Caldor!” -0.19444444444444448
Robin yelled desperately. -0.6
It all made sudden, horrible sense. -0.5
But a terrifying roar drowned out Robin’s words. -0.55
Glancing down, Robin realized that his own situation was nearly as dangerous as Caldor’s. -0.05185185185185185
Robin could only stare in disbelief as the dragon’s wings unfolded overhead, stretching out to beat the air in a deadly rhythm. -0.1
called Caldor, still desperately clutching the creature’s neck. -0.6
He hit the ground, shaking and gasping for breath, as the terrifying beast rose up into the air and disappeared into the mist, taking Caldor with him. -0.2
Then, to his horrified eyes, an empty piece of armo

In [13]:
prior_to_topic_update = [id[0] for id in topic_sent_list]

In [14]:
def get_sig_topic_id(array, threshold):
    elems, count = np.unique(array, return_counts=True)
    count_sort_ind = np.argsort(-count)[0]
    if count[count_sort_ind] >= threshold:
        return elems[count_sort_ind]
    return -1

In [15]:
prev_id = topic_sent_list[0][0]
window_length = 4
sig_threshold = 2
for i, (curr_id, sent) in enumerate(topic_sent_list):
    if prev_id == curr_id:
        continue
    before_sents = topic_sent_list[max(0, (i-window_length)):i]
    before_ids = [t[0] for t in before_sents]

    after_sents = topic_sent_list[(i + 1) :min(len(topic_sent_list), (i + window_length))]
    after_ids = [t[0] for t in after_sents]

    before_mode = get_sig_topic_id(before_ids, min(sig_threshold, len(before_ids)))
    after_mode = get_sig_topic_id(after_ids,  min(sig_threshold, len(after_ids)))
    
    print(curr_id, before_ids, after_ids, before_mode, after_mode)
    if before_mode == after_mode and before_mode > 0:
        topic_sent_list[i][0] = before_mode
        curr_id = before_mode
    print(curr_id)
    prev_id = curr_id
    # break
#     if(topic_sent_list[idx][0] != topic_sent_list[idx+1][0] and (topic_sent_list[idx][0] != topic_sent_list[idx-1][0])):  #start checking next elements
#         print("**",idx, sent)
#         curr_id = sent[0]
#     else:    
        
#         print(idx, sent)

7 [1] [4, 1, 1] 1 1
1
4 [1, 1] [1, 1, 5] 1 1
1
5 [1, 1, 1, 1] [16, 2, 2] 1 2
5
16 [1, 1, 1, 5] [2, 2, 8] 1 2
16
2 [1, 1, 5, 16] [2, 8, 2] 1 2
2
8 [5, 16, 2, 2] [2, 2, 7] 2 2
2
7 [2, 2, 2, 2] [19, 18, 12] 2 -1
7
19 [2, 2, 2, 7] [18, 12, 2] 2 -1
19
18 [2, 2, 7, 19] [12, 2, 2] 2 2
2
12 [2, 7, 19, 2] [2, 2, 2] 2 2
2
14 [2, 2, 2, 2] [2, 14, 12] 2 -1
14
2 [2, 2, 2, 14] [14, 12, 2] 2 -1
2
14 [2, 2, 14, 2] [12, 2, 2] 2 2
2
12 [2, 14, 2, 2] [2, 2, 2] 2 2
2
19 [2, 2, 2, 2] [4, 2, 2] 2 2
2
4 [2, 2, 2, 2] [2, 2, 15] 2 2
2
15 [2, 2, 2, 2] [2, 2, 2] 2 2
2
0 [2, 2, 2, 2] [2, 2, 2] 2 2
2
14 [2, 2, 2, 2] [6, 2, 2] 2 2
2
6 [2, 2, 2, 2] [2, 2, 16] 2 2
2
16 [2, 2, 2, 2] [11, 2, 2] 2 2
2
11 [2, 2, 2, 2] [2, 2, 2] 2 2
2
12 [2, 2, 2, 2] [2, 8, 2] 2 2
2
8 [2, 2, 2, 2] [2, 12, 2] 2 2
2
12 [2, 2, 2, 2] [2, 8, 4] 2 -1
12
2 [2, 2, 2, 12] [8, 4, 12] 2 -1
2
8 [2, 2, 12, 2] [4, 12, 12] 2 12
8
4 [2, 12, 2, 8] [12, 12, 4] 2 12
4
12 [12, 2, 8, 4] [12, 4, 8] -1 -1
12
4 [8, 4, 12, 12] [8, 12, 12] 12 12
12
8 [4, 12, 12, 1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
after_topic_update = [id[0] for id in topic_sent_list]

In [ ]:
for i in range(len(prior_to_topic_update)):
    print(prior_to_topic_update[i], after_topic_update[i])

In [ ]:
before_sents = topic_sent_list[max(0, (i-window_length)):i]
before_ids = [t[0] for t in before_sents]

after_sents = topic_sent_list[(i + 1) :min(len(topic_sent_list), (i + window_length))]
after_ids = [t[0] for t in after_sents]

before_mode = -1 
if len(scipy.stats.mode(before_ids)[0]) > 0:
    before_mode = scipy.stats.mode(before_ids)[0][0]
    
after_mode = -1 
if len(scipy.stats.mode(after_ids)[0]) > 0:
    after_mode = scipy.stats.mode(after_ids)[0][0]
    
if before_mode == after_mode:
    topic_sent_list[i][0] = before_mode
print(curr_id, before_ids, after_ids)
print(before_mode, after_mode)


In [ ]:



def contig_list_topic_id(array,topic_id):
    for idx,(id,sent) in  enumerate(array):
        cur_id = id
        cur_id_min_idx = idx
        if(cur_id == array[idx+1][0]):
            print('x')
        print('t')
        else:
            
    
    # for idx,(t_id,sent) in enumerate(array):
    #     if(topic_id ==t_id):
    #         print(idx,sent)
contig_list_topic_id(topic_sent_list,11)

In [ ]:
import more_itertools as mit
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]


iterable = [2, 3, 4, 5, 12, 13, 14, 15, 16, 17, 20]
list(find_ranges(iterable))


In [ ]:
[id[0] for id in topic_sent_list]